In [1]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [2]:
import sqlite3

In [3]:
import time

In [4]:
conn = sqlite3.connect('project_80pc.db')
cur = conn.cursor()

In [5]:
def add_tokens(sentence, t):
    s = sentence[0]
    text = sentence[1]
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        word = token.text.lower()
        pos = token.pos
        lemma = token.lemma
        
        token_id = t
        
        if pos == 'PUNCT':
            continue
        else:   
            t += 1
            cur.execute(
                    '''INSERT INTO main (token_id, sentence_id)
                        VALUES (?, ?)''',
                        (token_id, s)
                    )
            conn.commit()

            cur.execute(
                    '''INSERT INTO tokens (token_id, word, lemma, pos)
                        VALUES (?, ?, ?, ?)''',
                        (token_id, word, lemma, pos)
                    )
            conn.commit()
    return t

In [6]:
time1 = time.time()
t = 0
for i in range (0, 90220, 20):
    cur.execute("""
    select sentence_id, sentence from sentences
    where sentence_id >= ? and sentence_id < ?
    """, (i, i+20))
    sents = cur.fetchall()
    print(i)
    
    
    for sentence in sents:
        t = add_tokens(sentence, t)
        
        
time2 = time.time()

0
20
40
60
80
100
120
90000
90020
90040
90060
90080
90100
90120
90140
90160
90180
90200


In [7]:
print(time2-time1)

3977.3229410648346
